In [1]:
import glob
import pandas as pd
import numpy as np
df = None
for fname in glob.glob('res/results*.txt'):
    newdf = pd.read_csv(fname,sep=',', header=None,index_col=False,names=['sensitivity','ys_mean','ys_std','epsilon','steps','lengthscale','-','integral','-','cloaking'])
    if df is None:
        df = newdf
    else:
        df = pd.concat([df,newdf])
        #df.append(newdf, ignore_index=True)
#df.drop('-', axis=1, inplace=True)

df.loc[df['sensitivity']<0.01,'epsilon'] = 10000
#df = df[df['sensitivity']>0.001]
df['integral'] = df['integral'] * df['ys_std']
df['cloaking'] = df['cloaking'] * df['ys_std']

summary = {}
#nodp = df['no dp']
#summary['nodp'] = {'mean':np.mean(nodp), 'std':np.std(nodp), 'n':len(nodp)}
summary['cloaking'] = {}
summary['integral'] = {}

for eps in df['epsilon'].unique():
    summary['cloaking'][eps] = {}
    summary['integral'][eps] = {}
    for ls in df['lengthscale'].unique():
        cloaking = df[(df['epsilon']==eps) & (df['lengthscale']==ls)]['cloaking']
        summary['cloaking'][eps][ls] = {'mean':np.mean(cloaking), 'std':np.std(cloaking), 'n':len(cloaking)}
    for step in df['steps'].unique():
        integral = df[(df['epsilon']==eps) & (df['steps']==step)]['integral']
        summary['integral'][eps][step] = {'mean':np.mean(integral), 'std':np.std(integral), 'n':len(integral)}

In [2]:
def add_ste(s):
    s['ste'] = s['std']/np.sqrt(s['n'])
    
for k in summary['cloaking']:
    for l in summary['cloaking'][k]:
        add_ste(summary['cloaking'][k][l])
        
for k in summary['integral']:
    for l in summary['integral'][k]:
        add_ste(summary['integral'][k][l])

#add_ste(summary['nodp'])
        


In [5]:
# values from simple histogram data
means = np.array([[  542.67685719,   573.53396497,   704.89971696,  1344.05599004], [576.89537908,  1173.12983334,  2009.2342494 ,  4495.18246698],  [590.28755099,  1932.00996766,  3943.30129672,  8932.29461108]])
stes = np.array([[   6.82008082,    8.30943982,   46.61052769,  117.33564275],[ 7.92886834,   60.26630375,  166.87342616,  335.84487673], [ 7.38250292,   84.90635307,  198.94210199,  466.69851184]])

In [11]:
prepend = "\\begin{tabular}{ l l l l l l }\n"
prepend += "\\hline\n"
prepend += " &  & No DP & $\\varepsilon = 1$ & $\\varepsilon = 0.5$ & $\\varepsilon = 0.2$ \\\\\n"
prepend += " \\hline\n"
#prepend += " &  "
#for step in np.sort(df['steps'].unique()):
#    if step==1:
#        add_s = ""
#    else:
#        add_s = "s"
#    prepend += "& %d bin%s " % (step,add_s)
#prepend += "\\\\ \n"
#prepend += "\\hline\n"
append = "\\hline\n"
append += "\\end{tabular}\n"
rows = []
firstrow = True
for ls in np.sort(df['lengthscale'].unique())[::-1]:
    row = ""
    if firstrow:
        row += "cloaking "
        firstrow = False
    else:
        row += " "
    row += "& l=%0.3f " % (ls)
    for eps in np.sort(df['epsilon'].unique())[::-1]:
        row += " & $%0.0f \\pm %0.0f$" % (summary['cloaking'][eps][ls]['mean'], 1.96*summary['cloaking'][eps][ls]['ste'])
    row += " \\\\"
    rows.append(row)
    
rows.append(" \\hline")
    
firstrow = True
for steps in np.sort(df['steps'].unique())[::-1]:
    row = ""
    if firstrow:
        row += "integral binning "
        firstrow = False
    else:
        row += " "
    row += "& $%d^4$ bins " % (steps)
    for eps in np.sort(df['epsilon'].unique())[::-1]:
        row += " & $%0.0f \\pm %0.0f$" % (summary['integral'][eps][steps]['mean'], 1.96*summary['integral'][eps][steps]['ste'])
    row += " \\\\"
    rows.append(row)    
    
rows.append(" \\hline")
    
firstrow = True
i = 2
for steps in np.sort(df['steps'].unique())[::-1]:
    j = 0
    row = ""
    if firstrow:
        row += "simple binning "
        firstrow = False
    else:
        row += " "
    row += "& $%d^4$ bins " % (steps)
    for eps in np.sort(df['epsilon'].unique())[::-1]:
        row += " & $%0.0f \\pm %0.0f$" % (means[i][j],1.96*stes[i][j]) #these values are luckily in the right order to do this!
        j+=1
    row += " \\\\"
    i-=1
    rows.append(row)        

output = prepend + "\n".join(rows) + "\n" + append
 
print output    

\begin{tabular}{ l l l l l l }
\hline
 &  & No DP & $\varepsilon = 1$ & $\varepsilon = 0.5$ & $\varepsilon = 0.2$ \\
 \hline
cloaking & l=0.125  & $402 \pm 7$ & $437 \pm 21$ & $476 \pm 17$ & $758 \pm 94$ \\
 & l=0.050  & $333 \pm 11$ & $434 \pm 27$ & $612 \pm 78$ & $1163 \pm 147$ \\
 & l=0.020  & $314 \pm 12$ & $478 \pm 22$ & $854 \pm 54$ & $1868 \pm 106$ \\
 \hline
integral binning & $10^4$ bins  & $580 \pm 9$ & $599 \pm 23$ & $608 \pm 39$ & $882 \pm 111$ \\
 & $6^4$ bins  & $642 \pm 7$ & $652 \pm 29$ & $726 \pm 59$ & $948 \pm 118$ \\
 & $3^4$ bins  & $639 \pm 8$ & $687 \pm 47$ & $756 \pm 86$ & $1109 \pm 143$ \\
 \hline
simple binning & $10^4$ bins  & $590 \pm 14$ & $1932 \pm 166$ & $3943 \pm 390$ & $8932 \pm 915$ \\
 & $6^4$ bins  & $577 \pm 16$ & $1173 \pm 118$ & $2009 \pm 327$ & $4495 \pm 658$ \\
 & $3^4$ bins  & $543 \pm 13$ & $574 \pm 16$ & $705 \pm 91$ & $1344 \pm 230$ \\
\hline
\end{tabular}



In [56]:
eps

10

In [85]:
prepend = "\\begin{tabular}{ l l l l l }\n"
prepend += "\\hline\n"
prepend += "Epsilon & Cloaking & \\multicolumn{3}{ c }{Binning with Integral Kernel} \\\\\n"
prepend += " &  "
for step in np.sort(df['steps'].unique()):
    if step==1:
        add_s = ""
    else:
        add_s = "s"
    prepend += "& %d bin%s " % (step,add_s)
prepend += "\\\\ \n"
prepend += "\\hline\n"
append = "\\hline\n"
append += "\\end{tabular}\n"
rows = []
for ls in np.sort(df['lengthscale'].unique())[-1::-1]:
    rows.append("\\multicolumn{5}{ c }{Lengthscale = %0.4f} \\\\" % ls)
    for eps in np.sort(df['epsilon'].unique())[-1::-1]:
        row = ""
        row += "$%0.1f$ & $%0.1f \\pm %0.1f$ " % (eps, summary['cloaking'][eps][ls]['mean'], summary['cloaking'][eps][ls]['ste'])
        for step in np.sort(df['steps'].unique()):
            row += "& $%0.1f \\pm %0.1f$ " % (summary['integral'][eps][ls][step]['mean'], summary['integral'][eps][ls][step]['ste'])

            
            
        row += " \\\\"
        rows.append(row)

output = prepend + "\n".join(rows) + "\n" + append
 
print output    

\begin{tabular}{ l l l l l }
\hline
Epsilon & Cloaking & \multicolumn{3}{ c }{Binning with Integral Kernel} \\
 &  & 3 bins & 6 bins & 10 bins \\ 
\hline
\multicolumn{5}{ c }{Lengthscale = 0.1250} \\
$1.0$ & $437.2 \pm 10.7$ & $641.4 \pm 43.4$ & $648.3 \pm 29.2$ & $626.7 \pm 18.3$  \\
$0.5$ & $475.9 \pm 8.6$ & $784.1 \pm 93.7$ & $668.3 \pm 56.3$ & $599.5 \pm 40.4$  \\
$0.2$ & $758.4 \pm 48.1$ & $1171.9 \pm 111.5$ & $1098.4 \pm 133.8$ & $937.1 \pm 96.9$  \\
\multicolumn{5}{ c }{Lengthscale = 0.0500} \\
$1.0$ & $433.6 \pm 13.9$ & $734.8 \pm 41.0$ & $643.1 \pm 22.8$ & $585.3 \pm 18.6$  \\
$0.5$ & $612.5 \pm 39.8$ & $742.4 \pm 75.7$ & $757.3 \pm 52.5$ & $633.7 \pm 34.1$  \\
$0.2$ & $1163.4 \pm 74.8$ & $1132.9 \pm 158.6$ & $880.6 \pm 70.9$ & $877.7 \pm 113.5$  \\
\multicolumn{5}{ c }{Lengthscale = 0.0200} \\
$1.0$ & $478.0 \pm 11.4$ & $685.8 \pm 35.9$ & $665.6 \pm 24.1$ & $586.7 \pm 20.8$  \\
$0.5$ & $853.6 \pm 27.4$ & $741.9 \pm 57.1$ & $751.6 \pm 42.7$ & $589.1 \pm 23.4$  \\
$0.2$ & $1868